<a href="https://colab.research.google.com/github/NC25/esp_net/blob/main/ASR_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependecy Setup

In [ ]:
!sudo apt-get install bc tree

In [ ]:
!cat /etc/os-release

In [ ]:
!git clone https://github.com/espnet/espnet

In [ ]:
!cd espnet; pip install -e .

Tutorial

In [ ]:
!sudo apt update

In [ ]:
!git clone https://github.com/AssemblyAI-Examples/intro-to-espnet.git

In [ ]:
cd intro-to-espnet

In [ ]:
!virtualenv venv -p /usr/bin/python3.6
!source venv/bin/activate
!pip install --no-cache-dir -r requirements.txt

In [ ]:
!python3 -m /content/intro-to-espnet/speech2text.py

Code

In [ ]:
import subprocess as s
import os
import string
import soundfile
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

Download pretrained model

In [ ]:
tag = "Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best"
d = ModelDownloader()
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cpu", #cuda if gpu
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

In [ ]:
#capitalize all the text and remove punctuation

In [ ]:
def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))





*   return transcripts given an audio filepath
*   soundfile.read() reads in our audio data, and then speech2text predicts the speech.
*    isolate the best prediction with nbests[0] and then return





In [ ]:
def get_transcript(path):
    speech, rate = soundfile.read(path)
    nbests = speech2text(speech)
    text, *_ = nbests[0]
    return text, rate

Convert raw file into wave file, since function runs with wav files

In [ ]:


path = os.path.join(os.getcwd(), 'egs')
files = os.listdir(path+'/audio')



for file in files:
    if not file.endswith('.wav'):
        os.chdir(path+'/audio')
        s.run(f"ffmpeg -i {file} {file.split('.')[0]}.wav", shell=True, check=True, universal_newlines=False)
        os.chdir('../..')
        file = file.split('.')[0]+'.wav'
        
        # Transcribe and delete generated file
        text, est_rate = get_transcript(f'{path}/audio/{file}')
        os.remove(f'{path}/audio/{file}')
    else:
        text, est_rate = get_transcript(f'{path}/audio/{file}')

In [ ]:
    #prepare actual transcript
    label_file = file.split('.')[0]+'.txt'
    with open(f'{path}/text/{label_file}', 'r') as f:
        true_text = f.readline()
    # Print true transcript and hypothesis
    print(f"\n\nReference text: {true_text}")
    print(f"ASR hypothesis: {text_normalizer(text)}\n\n")